In [1]:
# configurar as variáveis de ambiente
import os
import elasticsearch
from elasticsearch import Elasticsearch, helpers
 
# tornar o pyspark "importável"
import findspark
findspark.init('../../anaconda3/spark-2.4.4-bin-hadoop2.7')

In [4]:
# iniciar uma sessão local
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import functions as f
from pyspark.sql.window import Window as w
import pyodbc
import pandas as pd
import json

appName = "PySpark Test"
conf = SparkConf() \
    .setAppName(appName) \
    .setMaster("local") 
sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)
spark = sqlContext.sparkSession

In [202]:
es = Elasticsearch([{'host':'localhost','port':9200}])

In [55]:
es.indices.create(index='andre_elk',ignore=400)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'andre_elk'}

In [ ]:
res=es.indices.get_alias("*")
for Name in res:
    print(Name)

In [176]:
df_spark = spark.read.option("delimiter", ";").csv("/home/aluisf/estudos/dados/mvno_arquivos/*.txt", header=True)

In [192]:
df_pd = df_spark.select("CPF_CNPJ","NOME_DO_ASSINANTE","NUMERO_DO_TERMINAL","SITUACAO","DATA_INICIO_DO_VINCULO","DATA_FIN_DO_CANCELAMENTO").toPandas()
df_pd.columns

Index(['CPF_CNPJ', 'NOME_DO_ASSINANTE', 'NUMERO_DO_TERMINAL', 'SITUACAO',
       'DATA_INICIO_DO_VINCULO', 'DATA_FIN_DO_CANCELAMENTO'],
      dtype='object')

In [193]:
df_pd.isna().sum()

CPF_CNPJ                     15
NOME_DO_ASSINANTE           350
NUMERO_DO_TERMINAL            0
SITUACAO                      0
DATA_INICIO_DO_VINCULO        0
DATA_FIN_DO_CANCELAMENTO      0
dtype: int64

In [194]:
df_pd = df_pd.dropna(subset=["CPF_CNPJ"])

In [189]:
df_pd.shape

(10002, 6)

In [195]:
df_pd2 = df_pd.to_dict('records')

In [212]:
df_pd2[0]

{'CPF_CNPJ': '07900991441',
 'NOME_DO_ASSINANTE': 'SANDRA MONICA LOTERO DA SILVA ',
 'NUMERO_DO_TERMINAL': '5582988827917',
 'SITUACAO': 'ATIVA',
 'DATA_INICIO_DO_VINCULO': '2021/04/13',
 'DATA_FIN_DO_CANCELAMENTO': 'NULL'}

In [218]:
def generator(df_pd2):
    for c, line in enumerate(df_pd2):
        yield {
            "_index":"andre_elk",
            "_id":line.get("CPF_CNPJ",None),
            "_source":{
                "nome_do_assinante":line.get("NOME_DO_ASSINANTE",""),
                "numero_do_terminal":line.get("NUMERO_DO_TERMINAL",""),
                "situacao":line.get("SITUACAO",""),
                "data_inicio":line.get("DATA_INICIO_DO_VINCULO",""),
                "data_fim":line.get("DATA_FIN_DO_CANCELAMENTO","")
            }
        }
        raise StopIteration

In [210]:
elk_test = generator(df_pd2)

In [208]:
next(elk_test)

{'_index': 'andre_elk',
 '_id': '07900991441',
 '_source': {'nome_do_assinante': 'SANDRA MONICA LOTERO DA SILVA ',
  'numero_do_terminal': '5582988827917',
  'situacao': 'ATIVA',
  'data_inicio': '2021/04/13',
  'data_fim': 'NULL'}}

In [219]:
try:
    res = helpers.bulk(es, generator(df_pd2))
    print("Carregando")
except Exception as e:
    print(e)

Carregando
